# Machine Learning - Final Project

In [ ]:
import pandas as pd
import numpy as np
from sklearn import * 
import time   
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

##Get Data From CSV or DB

In [ ]:
train_data = pd.read_csv("application_train.csv")
test_data = pd.read_csv("application_test.csv")

In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [ ]:
train_data.info()

In [ ]:
test_data.info()

####Check Data Label Distribution

In [ ]:
train_data.count()

In [ ]:
test_data.count()

In [ ]:
train_data.isna().sum()

In [ ]:
test_data.isna().sum()

##Preprocessing
####Drop Column where Null > 60%

In [ ]:
# Drop column with 60% null value
train_data = train_data.drop(train_data.columns[train_data.isnull().mean()>.6],axis=1)
test_data = test_data.drop(test_data.columns[test_data.isnull().mean()>.6],axis=1)

In [ ]:
# Store Clean Data
# train_data.to_sql('application_train_clean', engine, if_exists='replace', index=False)
# test_data.to_sql('application_test_clean', engine, if_exists='replace', index=False)

In [ ]:
X = train_data.drop(columns=["TARGET"], axis=1)
y = train_data["TARGET"]
X_test = test_data

####Data Splitting

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3)

In [ ]:
# Numerical data
num=X.select_dtypes(exclude='object').columns


In [ ]:
# Categorical data
cat=X.select_dtypes(include='object').columns

####SimpleImputer

In [ ]:
impute = SimpleImputer(strategy='median')
Xnum = impute.fit_transform(X[num])

####OneHotEncoder

In [ ]:
encode = OneHotEncoder()
Xcat = encode.fit_transform(X[cat])

##Modelling

In [ ]:
from sklearn.linear_model import LogisticRegression
from imblearn.pipeline import Pipeline
from sklearn.metrics import classification_report

####Transformer Steps

In [ ]:
cat_trans = Pipeline([
    ('c_i', SimpleImputer(strategy="most_frequent")),
    ('c', OneHotEncoder(handle_unknown="ignore"))
])

num_trans = Pipeline([
    ('n', SimpleImputer(strategy='median'))
])

transformer = [
    ('c_t', cat_trans, cat),
    ('n_t', num_trans, num)
]

####Logistic Regression

In [ ]:

model_lr = Pipeline([
    ('pre', ColumnTransformer(transformers=transformer)),
    ('model', LogisticRegression())
])

In [ ]:
model_lr.fit(X_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('pre',
                 ColumnTransformer(transformers=[('c_t',
                                                  Pipeline(steps=[('c_i',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('c',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
       'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
       'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'O...
       'FLAG_DOCUMENT_15', 'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_17',
       'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20',
       'FLAG_DOCUMENT_21', 'AMT_REQ_CREDIT_BUREAU_HOUR',
       'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK',
       'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_RE

In [ ]:
#ML Model Report
report = classification_report(y_test, model_lr.predict(X_test))
print(report)


              precision    recall  f1-score   support

           0       0.92      1.00      0.96     52535
           1       0.00      0.00      0.00      4538

    accuracy                           0.92     57073
   macro avg       0.46      0.50      0.48     57073
weighted avg       0.85      0.92      0.88     57073



####ML Model Test

Test by using dataframe from application_test.csv data

####Prediction

In [ ]:
y_predict = model_lr.predict(X_test)

####Probability

In [ ]:
y_prob = model_lr.predict_proba(X_test)

In [ ]:
# ML Model Result DataFrame

ml_result = X_test[['SK_ID_CURR']]

ml_result['prediction_target'] = y_predict.tolist()
ml_result['status'] = "Not Accepted"

status = ml_result['prediction_target'] >= 1
ml_result.loc[status, 'status'] = 'Accepted'
ml_result['probability'] =  y_prob.tolist()
ml_result

<ipython-input-63-9f3b4d38bccc>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ml_result['prediction_target'] = y_predict.tolist()
<ipython-input-63-9f3b4d38bccc>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ml_result['status'] = "Not Accepted"
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https

,SK_ID_CURR,prediction_target,status,probability
188044,317990,0,Not Accepted,"[0.9149670653426192, 0.08503293465738085]"
2605,103042,0,Not Accepted,"[0.8792758244179346, 0.12072417558206534]"
147102,270555,0,Not Accepted,"[0.9634850102579063, 0.03651498974209362]"
117165,235873,0,Not Accepted,"[0.8850272374275409, 0.11497276257245914]"
139746,262042,0,Not Accepted,"[0.9325495294630359, 0.06745047053696401]"
...,...,...,...,...
185395,314910,0,Not Accepted,"[0.9335569942423741, 0.06644300575762585]"
12666,114764,0,Not Accepted,"[0.9236891245642214, 0.07631087543577861]"
70954,182291,0,Not Accepted,"[0.9337366518537376, 0.06626334814626239]"
12040,114039,0,Not Accepted,"[0.9359456579383477, 0.06405434206165225]"


In [ ]:
ml_result = ml_result.sort_values(by='SK_ID_CURR')

In [ ]:
print(ml_result)

        SK_ID_CURR  prediction_target        status  \
19          100023                  0  Not Accepted   
23          100027                  0  Not Accepted   
26          100031                  0  Not Accepted   
30          100035                  0  Not Accepted   
32          100037                  0  Not Accepted   
...            ...                ...           ...   
190228      320559                  0  Not Accepted   
190230      320563                  0  Not Accepted   
190234      320570                  0  Not Accepted   
190237      320573                  0  Not Accepted   
190241      320577                  0  Not Accepted   

                                      probability  
19       [0.879471512617842, 0.12052848738215807]  
23      [0.9458623139201714, 0.05413768607982859]  
26      [0.8770613296068024, 0.12293867039319757]  
30         [0.884686341017423, 0.115313658982577]  
32       [0.9020037257410513, 0.0979962742589487]  
...                        

In [ ]:
# Store ML Test Result
# ml_result.to_sql('home_credit_default_risk_application_ml_result', engine, if_exists='replace', index=False)